In [1]:
import os 
import tiktoken
import openai 
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

env_path = ".env"
load_dotenv(dotenv_path=env_path,override=True)
api_type = "azure"
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
api_version = '2023-07-01-preview'
deployment_name = os.environ.get("AZURE_DEPLOYMENT_NAME")

encoding = tiktoken.get_encoding('cl100k_base')

azure_llm = AzureChatOpenAI(
            deployment_name=deployment_name,
            temperature=0,
            max_tokens=None,
            api_version="2023-07-01-preview",
            azure_endpoint =endpoint,
            api_key=api_key,

        )
def get_completion(prompt, model=azure_llm):
    messages =[{"role":"user","content":prompt}]
    response=model.invoke(messages)
    return response.content

### 1.) Self-Consistency
- bir cok ayni mantikda soru cevap örnegi vererek bu sorular böyle cevaplanir few shot la ögretip daha sonra kendir sorumuzu sor.

The primary objective is to improve the simplistic greedy decoding often used in chain-of-thought (CoT) prompting. In this technique, several diverse reasoning paths are sampled through few-shot CoT. The responses generated from these paths are then evaluated, and the most consistent answer is selected. This method has a significant impact on enhancing the effectiveness of CoT prompting, particularly in tasks that require arithmetic and commonsense reasoning. 

Reference: https://arxiv.org/abs/2203.11171

![Self-Consistency](./images/self-consistency-cot.png)

non self-consistency

In [2]:
prompt = f"""
When I was 12 my sister was half my age. Now
I’m 70 how old is my sister?
"""

response = get_completion(prompt)
print(response)

When you were 12, your sister was 6 (half your age). Now that you are 70, your sister is 70 - 12 + 6 = 64 years old.


with self-consistency

In [3]:
prompt = f"""
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent $15. She has $8 left.
Q: When I was 12 my sister was half my age. Now I’m 70 how old is my sister?
A:
"""

response = get_completion(prompt)
print(response)

When you were 12, your sister was half your age, so she was 6 years old. The age difference between you and your sister is 6 years. Now that you are 70, your sister must be 70 - 6 = 64 years old. The answer is 64.


### 2.) Generated Knowledge Prompting

Reference: https://arxiv.org/pdf/2110.08387.pdf

![](./images/generated-knowledge-prompting.png)

In [6]:
prompt = f"""
Hangi sehir yüz ölcümü olarak daha büyük? Istanbul mu Konya mi?
"""

response = get_completion(prompt)
print(response)# yanlis cevap üretti

İstanbul, yüz ölçümü olarak Konya'dan daha büyüktür. İstanbul'un yüz ölçümü 5,343 km² iken, Konya'nın yüz ölçümü 38,257 km²'dir. Bu nedenle İstanbul, Konya'dan daha büyük bir şehirdir.


In [8]:
prompt = f"""
Generate some knowledge about the sizes of Konya and Istanbul:
"""

response = get_completion(prompt)
print(response)

- Konya is the seventh most populous city in Turkey, with a population of over 2.2 million people.
- Istanbul is the most populous city in Turkey, with a population of over 15 million people.
- Konya covers an area of 38,873 square kilometers, making it the largest city in Turkey by area.
- Istanbul covers an area of 5,343 square kilometers, making it the second largest city in Turkey by area.
- Konya is located in the central Anatolian region of Turkey, while Istanbul is located in the northwest of the country, on the European side.


In [10]:
prompt = f"""
- Konya is the seventh most populous city in Turkey, with a population of over 2.2 million people.
- Istanbul is the most populous city in Turkey, with a population of over 15 million people.
- Konya covers an area of 38,873 square kilometers, making it the largest city in Turkey by area.
- Istanbul covers an area of 5,343 square kilometers, making it the second largest city in Turkey by area.
- Konya is located in the central Anatolian region of Turkey, while Istanbul is located in the northwest of the country, on the European side.


Hangi sehir yüz ölcümü olarak daha büyük? Istanbul mu Konya mi?
"""

response = get_completion(prompt)
print(response)# dogru cevabi ilk önce konu ile ilgili bilgileri getirerek, bu kontex icinden cevaba zoom yapmasini sagladik. Ilk halinde yaptigi hesaplamada büyük ihtimal istanbul ve büyük kelimeleri non parametrik hafizasinda cok gectigi icin onlara attention yapti ve yüz ölcümüne yeterince attention yapamadi.

Konya, yüz ölçümü olarak daha büyük bir şehirdir.


### 3.) Tree of Thoughts (ToT)

References:
- https://arxiv.org/abs/2305.10601
- https://arxiv.org/abs/2305.08291

![](./images/tot.png)

In [11]:
prompt = f"""
Which city is larger in terms of area: Istanbul or Konya ?
"""

response = get_completion(prompt)
print(response)# yanlis cevap üretti

Istanbul is larger in terms of area than Konya. Istanbul covers an area of 5,343 square kilometers, while Konya covers an area of 38,873 square kilometers.


In [12]:
prompt = f"""
Imagine five different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
The question is...

Which city is larger in terms of area: Istanbul or Konya ?

Give me a final conclusion.
"""
#Prompt reference: https://github.com/dave1010/tree-of-thought-prompting

response = get_completion(prompt)
print(response)

Expert 1: First, I will look up the land area of Istanbul.
Expert 2: I will do the same for Konya.
Expert 3: Istanbul has a land area of 5,343 square kilometers.
Expert 4: Konya has a land area of 38,257 square kilometers.
Expert 5: So, Konya is larger in terms of area than Istanbul.

Final conclusion: Konya is larger in terms of area than Istanbul.


### 4.) Rephase and Respond (RaR)

References:
https://arxiv.org/abs/2311.04205

In [13]:
prompt = f"""
Paul is born on 20th of January 1990. Was Paul born in an even day?
"""

response = get_completion(prompt)
print(response)

Yes, 20th of January is an even day.


In [14]:
prompt = f"""
Paul is born on 20th of January 1990. Was Paul born in an even day?

Rephrase and expand the question, and respond.
"""

response = get_completion(prompt)
print(response)

Was Paul born on a day that is divisible by 2? 

Yes, Paul was born on the 20th of January, which is an even number and therefore divisible by 2. So, Paul was indeed born on an even day.


### 5.) Retrievel Augmented Generation

![](./images/rag.jpg)

Reference: https://arxiv.org/pdf/2005.11401.pdf